In [1]:
import os
import json
from moviepy import VideoFileClip
import random
import warnings
warnings.filterwarnings('ignore')

def extract_event_clips_moviepy():
    """
    Extract clips for multiple events ("Goal", "Red card", "Yellow card", "Direct free-kick", "Penalty")
    and generate random non-event clips with sliding window for "Red card" and "Penalty".
    """
    # Base directory containing season folders
    base_dir = "F:/AIM Lab/Experiment"
    clips_base_dir = os.path.join(base_dir, "sliding-window", "Clips")
    
    # Create base clips directory if it doesn't exist
    os.makedirs(clips_base_dir, exist_ok=True)
    
    # Event types to process
    event_types = ["Goal", "Red card", "Yellow card", "Direct free-kick", "Penalty"]
    event_dirs = {event: os.path.join(clips_base_dir, event.replace(" ", "_")) for event in event_types}
    random_clips_dir = os.path.join(clips_base_dir, "no_event")
    
    # Create directories for each event and random clips
    for event_dir in event_dirs.values():
        os.makedirs(event_dir, exist_ok=True)
    os.makedirs(random_clips_dir, exist_ok=True)
    
    # Counters for clips
    event_counters = {event: 1 for event in event_types}
    random_counter = 1
    
    # Iterate through season folders (e.g., 2014-2015, 2015-2014)
    for season_folder in os.listdir(base_dir):
        season_path = os.path.join(base_dir, season_folder)
        if os.path.isdir(season_path) and season_folder.startswith("20"):
            print(f"Processing season: {season_folder}")
            
            # Iterate through match folders
            for match_folder in os.listdir(season_path):
                match_path = os.path.join(season_path, match_folder)
                if os.path.isdir(match_path):
                    print(f"  Processing match: {match_folder}")
                    
                    # Look for Labels-v2.json file
                    labels_file = os.path.join(match_path, "Labels-v2.json")
                    video_file = os.path.join(match_path, "2_720p.mkv")
                    
                    if os.path.exists(labels_file) and os.path.exists(video_file):
                        # Extract events from second half
                        event_timestamps = extract_event_timestamps(labels_file, event_types)
                        
                        # Load video once for all clips from this match
                        try:
                            video = VideoFileClip(video_file)
                            video_duration = video.duration
                            
                            # Create clips for each event
                            for event, timestamps in event_timestamps.items():
                                for timestamp in timestamps:
                                    # Create sliding window clips for "Red card" and "Penalty"
                                    if event in ["Red card", "Penalty"]:
                                        create_event_clip_moviepy(
                                            video, 
                                            timestamp, 
                                            event_dirs[event], 
                                            f"{event_counters[event]}_{match_folder}_normal",
                                            event
                                        )
                                        create_event_clip_moviepy(
                                            video, 
                                            timestamp - 2000,  # 2 seconds before
                                            event_dirs[event], 
                                            f"{event_counters[event]}_{match_folder}_before",
                                            event
                                        )
                                        create_event_clip_moviepy(
                                            video, 
                                            timestamp + 2000,  # 2 seconds after
                                            event_dirs[event], 
                                            f"{event_counters[event]}_{match_folder}_after",
                                            event
                                        )
                                    else:
                                        # Create normal clip for other events
                                        create_event_clip_moviepy(
                                            video, 
                                            timestamp, 
                                            event_dirs[event], 
                                            f"{event_counters[event]}_{match_folder}",
                                            event
                                        )
                                    event_counters[event] += 1
                            
                            # Generate random non-event clips
                            avoided_intervals = generate_avoided_intervals(event_timestamps)
                            num_random_clips = min(3, sum(len(timestamps) for timestamps in event_timestamps.values()))
                            random_start_times = generate_random_segments(video_duration, avoided_intervals, num_random_clips, 7.0)
                            
                            for start_time in random_start_times:
                                create_random_clip_moviepy(
                                    video, 
                                    start_time, 
                                    random_clips_dir, 
                                    f"ng{random_counter}_{match_folder}"
                                )
                                random_counter += 1
                            
                            # Close video to free memory
                            video.close()
                            
                        except Exception as e:
                            print(f"    Error loading video: {e}")
                    else:
                        if not os.path.exists(labels_file):
                            print(f"    Labels-v2.json not found")
                        if not os.path.exists(video_file):
                            print(f"    2_720p.mkv not found")

def extract_event_timestamps(labels_file, event_types):
    """
    Extract timestamps for specified events from the second half of the match.
    
    Args:
        labels_file (str): Path to the Labels-v2.json file
        event_types (list): List of event types to extract
        
    Returns:
        dict: Dictionary with event types as keys and lists of timestamps as values
    """
    try:
        with open(labels_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        event_timestamps = {event: [] for event in event_types}
        
        for annotation in data.get('annotations', []):
            label = annotation.get('label')
            game_time = annotation.get('gameTime', '')
            
            if label in event_types and game_time.startswith('2 -'):
                position = int(annotation.get('position', 0))
                event_timestamps[label].append(position)
        
        return event_timestamps
        
    except Exception as e:
        print(f"    Error reading labels file: {e}")
        return {event: [] for event in event_types}

def create_event_clip_moviepy(video_clip, position_ms, output_dir, clip_name, event):
    """
    Create a 7-second video clip around the event time using MoviePy.
    
    Args:
        video_clip (VideoFileClip): Loaded video clip
        position_ms (int): Event position in milliseconds
        output_dir (str): Directory to save the clip
        clip_name (str): Name for the output clip
        event (str): Event type for logging
    """
    try:
        # Calculate start and end times (3 seconds before and after)
        start_seconds = max(0, (position_ms - 3000) / 1000.0)  # 3 seconds before, but not negative
        end_seconds = (position_ms + 4000) / 1000.0  # 4 seconds after (total 7 seconds)
        
        # Make sure we don't exceed video duration
        end_seconds = min(end_seconds, video_clip.duration)
        
        # Output file path
        output_file = os.path.join(output_dir, f"{clip_name}.mp4")
        
        print(f"    Creating clip {clip_name} for {event} at {start_seconds:.2f}s")
        
        # Extract the clip
        event_clip = video_clip.subclipped(start_seconds, end_seconds)
        
        # Write the clip to file
        event_clip.write_videofile(
            output_file, 
            audio_codec='aac',
            codec='libx264',
            logger=None  # Suppress moviepy logs
        )
        
        # Close the clip to free memory
        event_clip.close()
        
        print(f"    Successfully created {output_file}")
            
    except Exception as e:
        print(f"    Error creating clip {clip_name}: {e}")

def create_random_clip_moviepy(video_clip, start_time, output_dir, clip_name):
    """
    Create a random 7-second video clip.
    
    Args:
        video_clip (VideoFileClip): Loaded video clip
        start_time (float): Start time in seconds
        output_dir (str): Directory to save the clip
        clip_name (str): Name for the output clip
    """
    try:
        # Calculate end time
        end_time = start_time + 7.0  # 7 seconds duration
        
        # Make sure we don't exceed video duration
        end_time = min(end_time, video_clip.duration)
        
        # Output file path
        output_file = os.path.join(output_dir, f"{clip_name}.mp4")
        
        print(f"    Creating random clip {clip_name}")
        
        # Extract the clip
        random_clip = video_clip.subclipped(start_time, end_time)
        
        # Write the clip to file
        random_clip.write_videofile(
            output_file, 
            audio_codec='aac',
            codec='libx264',
            logger=None  # Suppress moviepy logs
        )
        
        # Close the clip to free memory
        random_clip.close()
        
        print(f"    Successfully created {output_file}")
            
    except Exception as e:
        print(f"    Error creating random clip {clip_name}: {e}")

def generate_avoided_intervals(event_timestamps):
    """
    Generate avoided intervals for random clip generation based on event timestamps.
    
    Args:
        event_timestamps (dict): Dictionary of event timestamps
        
    Returns:
        list: List of avoided intervals (start, end) in seconds
    """
    avoided_intervals = []
    for timestamps in event_timestamps.values():
        for position_ms in timestamps:
            start_seconds = max(0, (position_ms - 3000) / 1000.0)  # 3 seconds before
            end_seconds = (position_ms + 4000) / 1000.0  # 4 seconds after
            avoided_intervals.append((start_seconds, end_seconds))
    return avoided_intervals

def generate_random_segments(video_duration, avoided_intervals, num_segments, segment_duration):
    """
    Generate random segments avoiding specified intervals.
    
    Args:
        video_duration (float): Total duration of the video
        avoided_intervals (list): List of (start, end) intervals to avoid
        num_segments (int): Number of segments to generate
        segment_duration (float): Duration of each segment
        
    Returns:
        list: List of start times for random segments
    """
    random_segments = []
    max_attempts = 1000
    
    for _ in range(num_segments):
        attempts = 0
        while attempts < max_attempts:
            max_start = video_duration - segment_duration
            if max_start <= 0:
                break
            
            start_time = random.uniform(0, max_start)
            end_time = start_time + segment_duration
            
            # Check if this segment overlaps with any avoided interval
            overlap = False
            for avoid_start, avoid_end in avoided_intervals:
                if not (end_time <= avoid_start or start_time >= avoid_end):
                    overlap = True
                    break
            
            if not overlap:
                random_segments.append(start_time)
                break
            
            attempts += 1
        
        if attempts >= max_attempts:
            print(f"Warning: Could not find non-overlapping segment after {max_attempts} attempts")
    
    return random_segments

# Run the extraction
print("Multiclass Clip Extractor (MoviePy)")
print("===================================")

try:
    extract_event_clips_moviepy()
    print("\nProcessing complete!")
except ImportError:
    print("Error: MoviePy not installed. Install with: pip install moviepy")
except Exception as e:
    print(f"Error: {e}")

Multiclass Clip Extractor (MoviePy)
Processing season: 2014-2015
  Processing match: 2015-02-21 - 18-00 Chelsea 1 - 1 Burnley
    Creating clip 1_2015-02-21 - 18-00 Chelsea 1 - 1 Burnley for Goal at 2118.49s
    Successfully created F:/AIM Lab/Experiment\sliding-window\Clips\Goal\1_2015-02-21 - 18-00 Chelsea 1 - 1 Burnley.mp4
    Creating clip 1_2015-02-21 - 18-00 Chelsea 1 - 1 Burnley_normal for Red card at 1462.20s
Proc not detected
    Successfully created F:/AIM Lab/Experiment\sliding-window\Clips\Red_card\1_2015-02-21 - 18-00 Chelsea 1 - 1 Burnley_normal.mp4
    Creating clip 1_2015-02-21 - 18-00 Chelsea 1 - 1 Burnley_before for Red card at 1460.20s
Proc not detected
    Successfully created F:/AIM Lab/Experiment\sliding-window\Clips\Red_card\1_2015-02-21 - 18-00 Chelsea 1 - 1 Burnley_before.mp4
    Creating clip 1_2015-02-21 - 18-00 Chelsea 1 - 1 Burnley_after for Red card at 1464.20s
Proc not detected
    Successfully created F:/AIM Lab/Experiment\sliding-window\Clips\Red_card\1

In [2]:
import os
import cv2
import numpy as np

def process_video(input_path, output_path):
    """
    Process the video to:
    1. Convert to grayscale.
    2. Remove grass and audience areas while preserving the ball.
    """
    # Load video
    cap = cv2.VideoCapture(input_path)
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Changed to mp4v for better compatibility
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height), isColor=False)
    
    print(f"Processing {total_frames} frames from {input_path}...")
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count}/{total_frames} frames")

        # Convert to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Convert original frame to HSV for better color detection
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Define green grass color range in HSV (more specific range)
        lower_green = np.array([35, 40, 40])
        upper_green = np.array([85, 255, 255])
        grass_mask = cv2.inRange(hsv, lower_green, upper_green)

        # Define ball detection (white/light colored ball)
        lower_ball_hsv = np.array([0, 0, 200])
        upper_ball_hsv = np.array([180, 30, 255])
        ball_mask_hsv = cv2.inRange(hsv, lower_ball_hsv, upper_ball_hsv)
        
        _, ball_mask_gray = cv2.threshold(gray_frame, 200, 255, cv2.THRESH_BINARY)
        ball_mask = cv2.bitwise_or(ball_mask_hsv, ball_mask_gray)
        
        kernel = np.ones((3, 3), np.uint8)
        ball_mask = cv2.morphologyEx(ball_mask, cv2.MORPH_CLOSE, kernel)
        ball_mask = cv2.morphologyEx(ball_mask, cv2.MORPH_OPEN, kernel)

        lower_audience = np.array([0, 100, 150])
        upper_audience = np.array([180, 255, 255])
        audience_mask = cv2.inRange(hsv, lower_audience, upper_audience)
        audience_mask = cv2.bitwise_and(audience_mask, cv2.bitwise_not(ball_mask))

        combined_mask = cv2.bitwise_or(grass_mask, audience_mask)
        keep_mask = cv2.bitwise_not(combined_mask)
        keep_mask = cv2.bitwise_or(keep_mask, ball_mask)

        processed_frame = cv2.bitwise_and(gray_frame, gray_frame, mask=keep_mask)
        out.write(processed_frame)
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Processed video saved as: {output_path}")

def process_all_videos(input_folder, output_folder):
    """
    Process all videos in the input folder and save them to the output folder.
    
    Args:
        input_folder (str): Path to the folder containing input videos.
        output_folder (str): Path to the folder to save processed videos.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for video_file in os.listdir(input_folder):
        input_path = os.path.join(input_folder, video_file)
        output_path = os.path.join(output_folder, video_file)
        
        if os.path.isfile(input_path) and video_file.endswith(".mp4"):
            process_video(input_path, output_path)

def process_all_events(base_input_path, base_output_path, event_types):
    """
    Process all event folders and save processed videos to respective folders.
    
    Args:
        base_input_path (str): Path to the base input folder containing event folders.
        base_output_path (str): Path to the base output folder to save processed videos.
        event_types (list): List of event types to process.
    """
    for event in event_types:
        input_folder = os.path.join(base_input_path, event.replace(" ", "_"))
        output_folder = os.path.join(base_output_path, event.replace(" ", "_"))
        
        if os.path.exists(input_folder):
            print(f"Processing videos for event: {event}")
            process_all_videos(input_folder, output_folder)
        else:
            print(f"Input folder not found for event: {event}")

if __name__ == "__main__":
    # Define input and output base paths
    base_input_path = "F:/AIM Lab/Experiment/sliding-window/Clips"
    base_output_path = "F:/AIM Lab/Experiment/sliding-window/Features-processed"
    
    # List of event types to process
    event_types = ["Goal", "Red card", "Yellow card", "Direct free-kick", "Penalty", "no_event"]
    
    # Process all events
    process_all_events(base_input_path, base_output_path, event_types)

Processing videos for event: Goal
Processing 175 frames from F:/AIM Lab/Experiment/sliding-window/Clips\Goal\10_2015-08-29 - 17-00 Manchester City 2 - 0 Watford.mp4...
Processed 100/175 frames
Processed video saved as: F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\10_2015-08-29 - 17-00 Manchester City 2 - 0 Watford.mp4
Processing 175 frames from F:/AIM Lab/Experiment/sliding-window/Clips\Goal\11_2015-09-12 - 14-45 Everton 3 - 1 Chelsea.mp4...
Processed 100/175 frames
Processed video saved as: F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\11_2015-09-12 - 14-45 Everton 3 - 1 Chelsea.mp4
Processing 175 frames from F:/AIM Lab/Experiment/sliding-window/Clips\Goal\12_2015-09-12 - 17-00 Crystal Palace 0 - 1 Manchester City.mp4...
Processed 100/175 frames
Processed video saved as: F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\12_2015-09-12 - 17-00 Crystal Palace 0 - 1 Manchester City.mp4
Processing 175 frames from F:/AIM Lab/Experiment/sliding-windo

In [3]:
import os
import shutil
import random
from collections import defaultdict

def split_videos(base_path, event_types, train_ratio=0.8, test_ratio=0.2):
    """
    Split videos into train and test subfolders for each event type based on unequal sample sizes.
    For "Red card" and "Penalty", ensure all three samples of an event (normal, before, after) remain together.
    
    Args:
        base_path (str): Path to the base folder containing event folders.
        event_types (list): List of event types to process.
        train_ratio (float): Proportion of videos to place in the train folder.
        test_ratio (float): Proportion of videos to place in the test folder.
    """
    for event in event_types:
        event_folder = os.path.join(base_path, event.replace(" ", "_"))
        train_folder = os.path.join(event_folder, "train")
        test_folder = os.path.join(event_folder, "test")
        
        # Create train and test subfolders if they don't exist
        os.makedirs(train_folder, exist_ok=True)
        os.makedirs(test_folder, exist_ok=True)
        
        # Get all video files in the event folder
        video_files = [f for f in os.listdir(event_folder) if f.endswith(".mp4")]
        
        if event in ["Red card", "Penalty"]:
            # Group files by event (normal, before, after)
            grouped_files = defaultdict(list)
            for filename in video_files:
                # Extract the base event identifier (e.g., "1_match_normal", "1_match_before", "1_match_after")
                base_event = "_".join(filename.split("_")[:2])  # First two parts of the filename
                grouped_files[base_event].append(filename)
            
            # Shuffle the grouped events to ensure randomness
            grouped_events = list(grouped_files.items())
            random.shuffle(grouped_events)
            
            # Calculate the number of train and test events
            total_events = len(grouped_events)
            num_train_events = int(total_events * train_ratio)
            num_test_events = total_events - num_train_events
            
            # Split events into train and test sets
            train_events = grouped_events[:num_train_events]
            test_events = grouped_events[num_train_events:]
            
            # Move grouped files to train folder
            for base_event, files in train_events:
                for file in files:
                    src_path = os.path.join(event_folder, file)
                    dest_path = os.path.join(train_folder, file)
                    shutil.move(src_path, dest_path)
            
            # Move grouped files to test folder
            for base_event, files in test_events:
                for file in files:
                    src_path = os.path.join(event_folder, file)
                    dest_path = os.path.join(test_folder, file)
                    shutil.move(src_path, dest_path)
            
            print(f"Event: {event}")
            print(f"  Total events: {total_events}")
            print(f"  Train events: {len(train_events)}")
            print(f"  Test events: {len(test_events)}")
        else:
            # Shuffle the video files to ensure randomness
            random.shuffle(video_files)
            
            # Calculate the number of train and test videos
            total_videos = len(video_files)
            num_train_videos = int(total_videos * train_ratio)
            num_test_videos = total_videos - num_train_videos
            
            # Split videos into train and test sets
            train_videos = video_files[:num_train_videos]
            test_videos = video_files[num_train_videos:]
            
            # Move videos to train folder
            for video in train_videos:
                src_path = os.path.join(event_folder, video)
                dest_path = os.path.join(train_folder, video)
                shutil.move(src_path, dest_path)
            
            # Move videos to test folder
            for video in test_videos:
                src_path = os.path.join(event_folder, video)
                dest_path = os.path.join(test_folder, video)
                shutil.move(src_path, dest_path)
            
            print(f"Event: {event}")
            print(f"  Total videos: {total_videos}")
            print(f"  Train videos: {len(train_videos)}")
            print(f"  Test videos: {len(test_videos)}")

if __name__ == "__main__":
    # Define base paths for Clips and Features-processed
    base_paths = [
        "F:/AIM Lab/Experiment/sliding-window/Clips",
        "F:/AIM Lab/Experiment/sliding-window/Features-processed"
    ]
    
    # List of event types to process
    event_types = ["Goal", "Red card", "Yellow card", "Direct_free-kick", "Penalty", "no_event"]
    
    # Split videos in both base paths
    for base_path in base_paths:
        split_videos(base_path, event_types)

Event: Goal
  Total videos: 37
  Train videos: 29
  Test videos: 8
Event: Red card
  Total events: 3
  Train events: 2
  Test events: 1
Event: Yellow card
  Total videos: 61
  Train videos: 48
  Test videos: 13
Event: Direct_free-kick
  Total videos: 63
  Train videos: 50
  Test videos: 13
Event: Penalty
  Total events: 5
  Train events: 4
  Test events: 1
Event: no_event
  Total videos: 69
  Train videos: 55
  Test videos: 14
Event: Goal
  Total videos: 37
  Train videos: 29
  Test videos: 8
Event: Red card
  Total events: 3
  Train events: 2
  Test events: 1
Event: Yellow card
  Total videos: 61
  Train videos: 48
  Test videos: 13
Event: Direct_free-kick
  Total videos: 63
  Train videos: 50
  Test videos: 13
Event: Penalty
  Total events: 5
  Train events: 4
  Test events: 1
Event: no_event
  Total videos: 69
  Train videos: 55
  Test videos: 14


In [1]:
import os
import cv2
import numpy as np
import torch
import torchvision
from typing import List, Dict
import random
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

# Load ResNet-50 model
model = torchvision.models.resnet50(pretrained=True)
model.fc = torch.nn.Identity()  # Remove the final classifier to get 2048-d features
model.eval()  # Set the model to evaluation mode

# Preprocessing pipeline for frames
preprocess = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),                    # H×W×C → C×H×W, [0,1]
    torchvision.transforms.Normalize(mean=[.485, .456, .406], std=[.229, .224, .225]),
    torchvision.transforms.Resize((224, 224)),
])

def embed_clip(frames: List[np.ndarray]):
    """
    Generate ResNet-50 embeddings for a list of frames.
    
    Args:
        frames (List[np.ndarray]): List of frames (H×W×C, BGR format).
    
    Returns:
        np.ndarray: N×2048 array of frame embeddings.
    """
    feats = []
    for frame in frames:
        # Convert BGR (OpenCV) to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        x = preprocess(frame_rgb).unsqueeze(0)  # 1×3×224×224
        with torch.no_grad():
            f = model(x)  # 1×2048
        feats.append(f.squeeze(0).cpu().numpy())
    return np.stack(feats, axis=0)  # N×2048

def sample_frames_from_video(video_path, num_samples=16):
    """
    Uniformly sample num_samples frames from the video at video_path.
    Returns a list of BGR frames (as numpy arrays).
    """
    print(f"Opening video: {video_path}")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video: {total_frames}")
    if total_frames == 0:
        print(f"Error: No frames found in video {video_path}")
        return []

    if total_frames < num_samples:
        # If fewer frames than samples, just read them all
        indices = list(range(total_frames))
    else:
        # Uniformly spaced frame indices
        indices = np.linspace(0, total_frames - 1, num=num_samples, dtype=int)

    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            print(f"Warning: Failed to read frame {idx} from {video_path}")
            continue
        frames.append(frame)
    cap.release()

    if not frames:
        print(f"Error: No frames sampled from video {video_path}")
    else:
        print(f"Successfully sampled {len(frames)} frames from {video_path}")
    return frames

def process_folder(input_folder, output_folder, num_samples=16):
    """
    Process videos in a folder, generate mean-pooled ResNet-50 embeddings for sampled frames,
    and save the embeddings to the output folder.
    
    Args:
        input_folder (str): Path to the folder containing input videos.
        output_folder (str): Path to save embeddings.
        num_samples (int): Number of frames to sample per video.
    """
    print(f"Processing folder: {input_folder}")
    if not os.path.exists(input_folder):
        print(f"Error: Folder does not exist: {input_folder}")
        return

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for fname in os.listdir(input_folder):
        if not fname.lower().endswith('.mp4'):
            print(f"Skipping non-video file: {fname}")
            continue
        video_path = os.path.join(input_folder, fname)
        print(f"Processing video: {video_path}")
        frames = sample_frames_from_video(video_path, num_samples=num_samples)
        if frames:
            # Generate embeddings and mean-pool them
            embeddings = embed_clip(frames)
            clip_vec = np.mean(embeddings, axis=0)  # Mean-pool → (2048,)
            
            # Save the embedding as a .npy file
            output_path = os.path.join(output_folder, f"{os.path.splitext(fname)[0]}.npy")
            np.save(output_path, clip_vec)
            print(f"Saved embedding to {output_path}")
        else:
            print(f"Error: No frames sampled from {fname}")

def l2_normalize(vec):
    """
    Perform L2 normalization on a vector.
    
    Args:
        vec (np.ndarray): Input vector.
    
    Returns:
        np.ndarray: L2-normalized vector.
    """
    norm = np.linalg.norm(vec)
    if norm == 0:
        return vec
    return vec / norm

def build_prototypes(embeddings_base_path, event_classes):
    """
    Build prototypes for each event class from training embeddings.
    
    Args:
        embeddings_base_path (str): Base path to embeddings folders.
        event_classes (list): List of event class names.
    
    Returns:
        dict: Dictionary of prototypes for each event class.
    """
    prototypes = {}
    
    for event in event_classes:
        event_train_folder = os.path.join(embeddings_base_path, event, "Train")
        
        if not os.path.exists(event_train_folder):
            print(f"Warning: Training folder not found for {event}")
            continue
            
        event_vecs = []
        print(f"Loading training embeddings for {event}...")
        
        # Load embeddings for the event
        for fname in os.listdir(event_train_folder):
            if fname.endswith(".npy"):
                vec = np.load(os.path.join(event_train_folder, fname))
                event_vecs.append(vec)
                print(f"  Loaded {fname} (shape: {vec.shape})")

        if event_vecs:
            # Compute mean vector for the event and normalize
            proto_event = np.mean(event_vecs, axis=0)
            proto_event = l2_normalize(proto_event)
            prototypes[event] = proto_event
            print(f"Built prototype for {event} from {len(event_vecs)} samples")
        else:
            print(f"Warning: No embeddings found for {event}")

    return prototypes

def classify_clip(clip_embedding, prototypes):
    """
    Classify a clip based on cosine similarity to prototypes.
    
    Args:
        clip_embedding (np.ndarray): Embedding vector of the clip.
        prototypes (dict): Dictionary of prototypes for each event class.
    
    Returns:
        tuple: (predicted_label, similarity_scores)
    """
    # Normalize the clip embedding
    clip_norm = l2_normalize(clip_embedding)
    
    # Compute cosine similarities
    similarities = {}
    for event, proto in prototypes.items():
        similarity = np.dot(clip_norm, proto)
        similarities[event] = similarity
    
    # Predict the class with highest similarity
    predicted_label = max(similarities, key=similarities.get)
    
    return predicted_label, similarities

def evaluate_multiclass_classification(embeddings_base_path, prototypes, event_classes):
    """
    Evaluate multiclass classification performance on test data.
    
    Args:
        embeddings_base_path (str): Base path to embeddings folders.
        prototypes (dict): Dictionary of prototypes for each event class.
        event_classes (list): List of event class names.
    
    Returns:
        dict: Dictionary containing evaluation results.
    """
    y_true = []
    y_pred = []
    results = {
        'predictions': [],
        'class_counts': {event: {'correct': 0, 'total': 0} for event in event_classes}
    }
    
    print("\n" + "="*60)
    print("EVALUATING MULTICLASS CLASSIFICATION")
    print("="*60)
    
    for event in event_classes:
        event_test_folder = os.path.join(embeddings_base_path, event, "Test")
        
        if not os.path.exists(event_test_folder):
            print(f"Warning: Test folder not found for {event}")
            continue
            
        print(f"\nTesting {event} clips...")
        
        for fname in os.listdir(event_test_folder):
            if fname.endswith(".npy"):
                test_embedding = np.load(os.path.join(event_test_folder, fname))
                predicted_label, similarities = classify_clip(test_embedding, prototypes)
                
                is_correct = predicted_label == event
                results['class_counts'][event]['total'] += 1
                
                if is_correct:
                    results['class_counts'][event]['correct'] += 1
                
                # Store for sklearn metrics
                y_true.append(event)
                y_pred.append(predicted_label)
                
                # Store detailed results
                result_entry = {
                    'file': fname,
                    'true_label': event,
                    'predicted_label': predicted_label,
                    'correct': is_correct,
                    'similarities': similarities
                }
                results['predictions'].append(result_entry)
                
                # Print results
                print(f"  File: {fname}")
                print(f"    True: {event}, Predicted: {predicted_label}, Correct: {is_correct}")
                print(f"    Similarities: {similarities}")
    
    # Calculate overall metrics
    overall_accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=event_classes)
    classification_rep = classification_report(y_true, y_pred, labels=event_classes, zero_division=0)
    
    # Calculate sklearn metrics for each class (treating as one-vs-rest)
    precision = precision_score(y_true, y_pred, labels=event_classes, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, labels=event_classes, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, labels=event_classes, average='weighted', zero_division=0)
    
    # Calculate per-class accuracy
    class_accuracies = {}
    for event in event_classes:
        if results['class_counts'][event]['total'] > 0:
            class_accuracies[event] = results['class_counts'][event]['correct'] / results['class_counts'][event]['total']
        else:
            class_accuracies[event] = 0.0
    
    # Print results
    print(f"\n{'='*60}")
    print("EVALUATION RESULTS")
    print(f"{'='*60}")
    print(f"Overall Accuracy: {overall_accuracy:.4f}")
    print(f"Weighted Precision: {precision:.4f}")
    print(f"Weighted Recall: {recall:.4f}")
    print(f"Weighted F₁ Score: {f1:.4f}")
    
    print(f"\nPer-class Accuracy:")
    for event, acc in class_accuracies.items():
        correct = results['class_counts'][event]['correct']
        total = results['class_counts'][event]['total']
        print(f"  {event}: {acc:.4f} ({correct}/{total})")
    
    print(f"\nConfusion Matrix:")
    print("Rows: True labels, Columns: Predicted labels")
    print(f"Classes: {event_classes}")
    print(cm)
    
    print(f"\nDetailed Classification Report:")
    print(classification_rep)
    
    results.update({
        'overall_accuracy': overall_accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'class_accuracies': class_accuracies,
        'confusion_matrix': cm,
        'classification_report': classification_rep,
        'y_true': y_true,
        'y_pred': y_pred
    })
    
    return results

def main():
    """
    Main function to run multiclass few-shot learning classification.
    """
    print("Multiclass Few-Shot Learning with ResNet-50")
    print("="*50)
    
    # Configuration
    base_input_path = "F:/AIM Lab/Experiment/sliding-window/Features-processed"
    base_output_path = "F:/AIM Lab/Experiment/sliding-window/Resnet-50 embeddings"
    
    # Event classes
    event_classes = ["Goal", "Red_card", "Yellow_card", "Direct_free-kick", "Penalty", "no_event"]
    
    # Step 1: Generate embeddings for all videos
    print("\nStep 1: Generating ResNet-50 embeddings...")
    for event in event_classes:
        for split in ["Train", "Test"]:
            input_folder = os.path.join(base_input_path, event, split)
            output_folder = os.path.join(base_output_path, event, split)
            
            print(f"\nProcessing {event} - {split}...")
            process_folder(input_folder, output_folder, num_samples=16)
    
    # Step 2: Build prototypes from training embeddings
    print("\nStep 2: Building prototypes from training data...")
    prototypes = build_prototypes(base_output_path, event_classes)
    
    if not prototypes:
        print("Error: No prototypes could be built!")
        return
    
    print(f"Successfully built prototypes for {len(prototypes)} classes:")
    for event, proto in prototypes.items():
        print(f"  {event}: shape {proto.shape}")
    
    # Step 3: Evaluate on test data
    print("\nStep 3: Evaluating classification performance...")
    results = evaluate_multiclass_classification(base_output_path, prototypes, event_classes)
    
    # Step 4: Additional analysis
    print(f"\nStep 4: Additional Analysis...")
    print(f"Total test samples: {len(results['y_true'])}")
    print(f"Number of classes: {len(event_classes)}")
    
    # Find most confused classes
    cm = results['confusion_matrix']
    print(f"\nMost confused class pairs:")
    for i, true_class in enumerate(event_classes):
        for j, pred_class in enumerate(event_classes):
            if i != j and cm[i, j] > 0:
                print(f"  {true_class} → {pred_class}: {cm[i, j]} times")
    
        print(f"\n{'='*50}")
        print("FINAL RESULTS")
        print(f"{'='*50}")
        print(f"Overall Accuracy: {results['overall_accuracy']:.4f}")
        print(f"Weighted Precision: {results['precision']:.4f}")
        print(f"Weighted Recall: {results['recall']:.4f}")
        print(f"Weighted F₁ Score: {results['f1_score']:.4f}")
        
        print(f"\n{'='*50}")
        print("MULTICLASS FEW-SHOT LEARNING COMPLETE!")
        print(f"{'='*50}")
    print(f"\n{'='*50}")
    print("MULTICLASS FEW-SHOT LEARNING COMPLETE!")
    print(f"{'='*50}")

if __name__ == "__main__":
    main()

c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Multiclass Few-Shot Learning with ResNet-50

Step 1: Generating ResNet-50 embeddings...

Processing Goal - Train...
Processing folder: F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\Train
Processing video: F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\Train\10_2015-08-29 - 17-00 Manchester City 2 - 0 Watford.mp4
Opening video: F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\Train\10_2015-08-29 - 17-00 Manchester City 2 - 0 Watford.mp4
Total frames in video: 175
Successfully sampled 16 frames from F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\Train\10_2015-08-29 - 17-00 Manchester City 2 - 0 Watford.mp4
Saved embedding to F:/AIM Lab/Experiment/sliding-window/Resnet-50 embeddings\Goal\Train\10_2015-08-29 - 17-00 Manchester City 2 - 0 Watford.npy
Processing video: F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\Train\11_2015-09-12 - 14-45 Everton 3 - 1 Chelsea.mp4
Opening video: F:/AIM Lab/Experiment/sliding-window/Feat

In [5]:
import os
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import transforms
from torchvision.models.video import r2plus1d_18
from typing import List, Dict
import random
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

class R2Plus1DFeatureExtractor:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        """
        Initialize R(2+1)D model for feature extraction.
        
        Args:
            device: Device to run the model on
        """
        self.device = device
        
        # Initialize R(2+1)D model
        self.model = r2plus1d_18(pretrained=True)
        
        # Remove the final classification layer to get features
        self.model.fc = torch.nn.Identity()
        self.model.to(device)
        self.model.eval()
        
        # Preprocessing transforms
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((112, 112)),  # R(2+1)D typically uses 112x112
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.43216, 0.394666, 0.37645], 
                               std=[0.22803, 0.22145, 0.216989])  # Kinetics normalization
        ])
    
    def preprocess_frames(self, frames: List[np.ndarray], num_frames: int = 16) -> torch.Tensor:
        """
        Preprocess frames for R(2+1)D input.
        
        Args:
            frames: List of frames (H×W×C, BGR format)
            num_frames: Number of frames to use for R(2+1)D
            
        Returns:
            Preprocessed tensor of shape (1, 3, num_frames, H, W)
        """
        # Sample frames uniformly if we have more than needed
        if len(frames) > num_frames:
            indices = np.linspace(0, len(frames) - 1, num_frames, dtype=int)
            frames = [frames[i] for i in indices]
        elif len(frames) < num_frames:
            # Repeat last frame if we don't have enough
            while len(frames) < num_frames:
                frames.append(frames[-1])
        
        # Convert frames and apply transforms
        processed_frames = []
        for frame in frames:
            # Convert BGR to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Apply transforms
            frame_tensor = self.transform(frame_rgb)
            processed_frames.append(frame_tensor)
        
        # Stack frames: (num_frames, 3, H, W) -> (3, num_frames, H, W)
        video_tensor = torch.stack(processed_frames, dim=1)  # (3, num_frames, H, W)
        video_tensor = video_tensor.unsqueeze(0)  # Add batch dimension
        
        return video_tensor
    
    def extract_features(self, frames: List[np.ndarray]) -> np.ndarray:
        """
        Extract R(2+1)D features from video frames.
        
        Args:
            frames: List of video frames
            
        Returns:
            Feature vector of shape (512,) for R(2+1)D-18
        """
        # Preprocess frames
        video_tensor = self.preprocess_frames(frames)
        video_tensor = video_tensor.to(self.device)
        
        # Extract features
        with torch.no_grad():
            features = self.model(video_tensor)
            # Global average pooling if needed
            if len(features.shape) > 2:
                features = F.adaptive_avg_pool3d(features, 1).squeeze()
            else:
                features = features.squeeze()
        
        return features.cpu().numpy()

def sample_frames_from_video(video_path: str, num_samples: int = 32) -> List[np.ndarray]:
    """
    Uniformly sample frames from a video.
    
    Args:
        video_path: Path to video file
        num_samples: Number of frames to sample
        
    Returns:
        List of sampled frames
    """
    print(f"Opening video: {video_path}")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video: {total_frames}")
    
    if total_frames == 0:
        print(f"Error: No frames found in video {video_path}")
        return []

    if total_frames < num_samples:
        indices = list(range(total_frames))
    else:
        indices = np.linspace(0, total_frames - 1, num=num_samples, dtype=int)

    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            print(f"Warning: Failed to read frame {idx} from {video_path}")
            continue
        frames.append(frame)
    
    cap.release()
    
    if not frames:
        print(f"Error: No frames sampled from video {video_path}")
    else:
        print(f"Successfully sampled {len(frames)} frames from {video_path}")
    
    return frames

def process_folder(input_folder: str, output_folder: str, 
                  feature_extractor: R2Plus1DFeatureExtractor,
                  num_frames: int = 32):
    """
    Process all videos in a folder and generate R(2+1)D embeddings.
    
    Args:
        input_folder: Folder containing input videos
        output_folder: Folder to save embeddings
        feature_extractor: R(2+1)D feature extractor instance
        num_frames: Number of frames to use for each video
    """
    print(f"Processing folder: {input_folder}")
    if not os.path.exists(input_folder):
        print(f"Error: Folder does not exist: {input_folder}")
        return

    os.makedirs(output_folder, exist_ok=True)

    for fname in os.listdir(input_folder):
        if not fname.lower().endswith('.mp4'):
            print(f"Skipping non-video file: {fname}")
            continue
        
        video_path = os.path.join(input_folder, fname)
        print(f"Processing video: {video_path}")
        
        # Sample frames from video
        frames = sample_frames_from_video(video_path, num_samples=num_frames)
        
        if frames:
            # Extract R(2+1)D features
            try:
                features = feature_extractor.extract_features(frames)
                
                # Save the embedding as a .npy file
                output_path = os.path.join(output_folder, f"{os.path.splitext(fname)[0]}.npy")
                np.save(output_path, features)
                print(f"Saved R(2+1)D embedding to {output_path} (shape: {features.shape})")
                
            except Exception as e:
                print(f"Error processing {fname}: {e}")
        else:
            print(f"Error: No frames sampled from {fname}")

def l2_normalize(vec: np.ndarray) -> np.ndarray:
    """
    Perform L2 normalization on a vector.
    
    Args:
        vec: Input vector
        
    Returns:
        L2-normalized vector
    """
    norm = np.linalg.norm(vec)
    if norm == 0:
        return vec
    return vec / norm

def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """
    Compute cosine similarity between two vectors.
    
    Args:
        vec1: First vector
        vec2: Second vector
        
    Returns:
        Cosine similarity value
    """
    # Normalize vectors
    vec1_norm = l2_normalize(vec1)
    vec2_norm = l2_normalize(vec2)
    
    # Compute cosine similarity
    return np.dot(vec1_norm, vec2_norm)

def build_prototypes(embeddings_base_path: str, event_classes: List[str]) -> Dict[str, np.ndarray]:
    """
    Build prototypes for each event class from training embeddings.
    
    Args:
        embeddings_base_path: Base path to embeddings folders
        event_classes: List of event class names
        
    Returns:
        Dictionary of prototypes for each event class
    """
    prototypes = {}
    
    for event in event_classes:
        event_train_folder = os.path.join(embeddings_base_path, event, "Train")
        
        if not os.path.exists(event_train_folder):
            print(f"Warning: Training folder not found for {event}")
            continue
            
        event_vecs = []
        print(f"Loading training embeddings for {event}...")
        
        # Load embeddings for the event
        for fname in os.listdir(event_train_folder):
            if fname.endswith(".npy"):
                vec = np.load(os.path.join(event_train_folder, fname))
                event_vecs.append(vec)
                print(f"  Loaded {fname} (shape: {vec.shape})")

        if event_vecs:
            # Compute mean vector for the event and normalize
            proto_event = np.mean(event_vecs, axis=0)
            proto_event = l2_normalize(proto_event)
            prototypes[event] = proto_event
            print(f"Built prototype for {event} from {len(event_vecs)} samples")
        else:
            print(f"Warning: No embeddings found for {event}")

    return prototypes

def classify_clip(clip_embedding: np.ndarray, prototypes: Dict[str, np.ndarray]) -> tuple:
    """
    Classify a clip based on cosine similarity to prototypes.
    
    Args:
        clip_embedding: Embedding vector of the clip
        prototypes: Dictionary of prototypes for each event class
    
    Returns:
        tuple: (predicted_label, similarity_scores)
    """
    # Normalize the clip embedding
    clip_norm = l2_normalize(clip_embedding)
    
    # Compute cosine similarities
    similarities = {}
    for event, proto in prototypes.items():
        similarity = cosine_similarity(clip_norm, proto)
        similarities[event] = similarity
    
    # Predict the class with highest similarity
    predicted_label = max(similarities, key=similarities.get)
    
    return predicted_label, similarities

def evaluate_multiclass_classification(embeddings_base_path: str, 
                                     prototypes: Dict[str, np.ndarray], 
                                     event_classes: List[str],
                                     threshold: float = 0.0) -> dict:
    """
    Evaluate multiclass classification performance on test data.
    
    Args:
        embeddings_base_path: Base path to embeddings folders
        prototypes: Dictionary of prototypes for each event class
        event_classes: List of event class names
        threshold: Decision threshold (not used in multiclass, kept for compatibility)
    
    Returns:
        Dictionary containing evaluation results
    """
    y_true = []
    y_pred = []
    results = {
        'predictions': [],
        'class_counts': {event: {'correct': 0, 'total': 0} for event in event_classes}
    }
    
    print("\n" + "="*60)
    print("EVALUATING MULTICLASS CLASSIFICATION")
    print("="*60)
    
    for event in event_classes:
        event_test_folder = os.path.join(embeddings_base_path, event, "Test")
        
        if not os.path.exists(event_test_folder):
            print(f"Warning: Test folder not found for {event}")
            continue
            
        print(f"\nTesting {event} clips...")
        
        for fname in os.listdir(event_test_folder):
            if fname.endswith(".npy"):
                test_embedding = np.load(os.path.join(event_test_folder, fname))
                predicted_label, similarities = classify_clip(test_embedding, prototypes)
                
                is_correct = predicted_label == event
                results['class_counts'][event]['total'] += 1
                
                if is_correct:
                    results['class_counts'][event]['correct'] += 1
                
                # Store for sklearn metrics
                y_true.append(event)
                y_pred.append(predicted_label)
                
                # Store detailed results
                result_entry = {
                    'file': fname,
                    'true_label': event,
                    'predicted_label': predicted_label,
                    'correct': is_correct,
                    'similarities': similarities
                }
                results['predictions'].append(result_entry)
                
                # Print results
                print(f"  File: {fname}")
                print(f"    True: {event}, Predicted: {predicted_label}, Correct: {is_correct}")
                print(f"    Similarities: {similarities}")
    
    # Calculate overall metrics
    overall_accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=event_classes)
    classification_rep = classification_report(y_true, y_pred, labels=event_classes, zero_division=0)
    
    # Calculate sklearn metrics for each class (treating as one-vs-rest)
    precision = precision_score(y_true, y_pred, labels=event_classes, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, labels=event_classes, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, labels=event_classes, average='weighted', zero_division=0)
    
    # Calculate per-class accuracy
    class_accuracies = {}
    for event in event_classes:
        if results['class_counts'][event]['total'] > 0:
            class_accuracies[event] = results['class_counts'][event]['correct'] / results['class_counts'][event]['total']
        else:
            class_accuracies[event] = 0.0
    
    # Print results
    print(f"\n{'='*60}")
    print("EVALUATION RESULTS")
    print(f"{'='*60}")
    print(f"Overall Accuracy: {overall_accuracy:.4f}")
    print(f"Weighted Precision: {precision:.4f}")
    print(f"Weighted Recall: {recall:.4f}")
    print(f"Weighted F₁ Score: {f1:.4f}")
    
    print(f"\nPer-class Accuracy:")
    for event, acc in class_accuracies.items():
        correct = results['class_counts'][event]['correct']
        total = results['class_counts'][event]['total']
        print(f"  {event}: {acc:.4f} ({correct}/{total})")
    
    print(f"\nConfusion Matrix:")
    print("Rows: True labels, Columns: Predicted labels")
    print(f"Classes: {event_classes}")
    print(cm)
    
    print(f"\nDetailed Classification Report:")
    print(classification_rep)
    
    results.update({
        'overall_accuracy': overall_accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'class_accuracies': class_accuracies,
        'confusion_matrix': cm,
        'classification_report': classification_rep,
        'y_true': y_true,
        'y_pred': y_pred
    })
    
    return results

def main():
    """
    Main function to run R(2+1)D-based multiclass few-shot learning classification.
    """
    print("Multiclass Few-Shot Learning with R(2+1)D")
    print("="*50)
    
    # Configuration
    base_input_path = "F:/AIM Lab/Experiment/sliding-window/Features-processed"
    base_output_path = "F:/AIM Lab/Experiment/sliding-window/R(2+1)D embeddings"
    
    # Event classes (using underscores to match folder names)
    event_classes = ["Goal", "Red_card", "Yellow_card", "Direct_free-kick", "Penalty", "no_event"]
    
    # Initialize R(2+1)D feature extractor
    print("Initializing R(2+1)D feature extractor...")
    feature_extractor = R2Plus1DFeatureExtractor()
    
    # Step 1: Generate R(2+1)D embeddings for all videos
    print("\nStep 1: Generating R(2+1)D embeddings...")
    for event in event_classes:
        for split in ["Train", "Test"]:
            input_folder = os.path.join(base_input_path, event, split)
            output_folder = os.path.join(base_output_path, event, split)
            
            print(f"\nProcessing {event} - {split}...")
            process_folder(input_folder, output_folder, feature_extractor, num_frames=32)
    
    # Step 2: Build prototypes from training embeddings
    print("\nStep 2: Building prototypes from training data...")
    prototypes = build_prototypes(base_output_path, event_classes)
    
    if not prototypes:
        print("Error: No prototypes could be built!")
        return
    
    print(f"Successfully built prototypes for {len(prototypes)} classes:")
    for event, proto in prototypes.items():
        print(f"  {event}: shape {proto.shape}")
    
    # Step 3: Evaluate on test data with threshold 0.0
    print("\nStep 3: Evaluating classification performance with threshold 0.0...")
    results = evaluate_multiclass_classification(base_output_path, prototypes, event_classes, threshold=0.0)
    
    # Step 4: Additional analysis
    print(f"\nStep 4: Additional Analysis...")
    print(f"Total test samples: {len(results['y_true'])}")
    print(f"Number of classes: {len(event_classes)}")
    
    # Find most confused classes
    cm = results['confusion_matrix']
    print(f"\nMost confused class pairs:")
    for i, true_class in enumerate(event_classes):
        for j, pred_class in enumerate(event_classes):
            if i != j and cm[i, j] > 0:
                print(f"  {true_class} → {pred_class}: {cm[i, j]} times")
    
    print(f"\n{'='*50}")
    print("FINAL RESULTS")
    print(f"{'='*50}")
    print(f"Overall Accuracy: {results['overall_accuracy']:.4f}")
    print(f"Weighted Precision: {results['precision']:.4f}")
    print(f"Weighted Recall: {results['recall']:.4f}")
    print(f"Weighted F₁ Score: {results['f1_score']:.4f}")
    
    print(f"\n{'='*50}")
    print("MULTICLASS FEW-SHOT LEARNING COMPLETE!")
    print(f"{'='*50}")

if __name__ == "__main__":
    main()

Multiclass Few-Shot Learning with R(2+1)D
Initializing R(2+1)D feature extractor...

Step 1: Generating R(2+1)D embeddings...

Processing Goal - Train...
Processing folder: F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\Train
Processing video: F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\Train\10_2015-08-29 - 17-00 Manchester City 2 - 0 Watford.mp4
Opening video: F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\Train\10_2015-08-29 - 17-00 Manchester City 2 - 0 Watford.mp4
Total frames in video: 175
Successfully sampled 32 frames from F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\Train\10_2015-08-29 - 17-00 Manchester City 2 - 0 Watford.mp4
Saved R(2+1)D embedding to F:/AIM Lab/Experiment/sliding-window/R(2+1)D embeddings\Goal\Train\10_2015-08-29 - 17-00 Manchester City 2 - 0 Watford.npy (shape: (512,))
Processing video: F:/AIM Lab/Experiment/sliding-window/Features-processed\Goal\Train\11_2015-09-12 - 14-45 Everton 3 - 1 Chelsea.